# Metadata

::: {.content-hidden when-format="html"}

## Project Setup

Install and load the necessary packages

In [13]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [14]:
#| echo: false
#| output: false
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


::: {.panel-tabset}

#### R

In [15]:
%%capture 
%%R

library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [16]:
%%capture --no-display --no-stdout
%%R

version <- packageVersion("REDCapR")
version

[1] '1.1.9005'


In this project, we will use the bleeding edge version of REDCapR available on Github

In [6]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] "REDCapR package up to date"


#### Python

In [17]:
import redcap
import json
import pandas as pd

:::

Assign your project URL and Token

::: {.panel-tabset}

#### R

In [18]:
%%R
path = paste0("C:/Users/", Sys.getenv("USERNAME"), '/json_api_data.json')
token <- jsonlite::fromJSON(path)$dev_token$'308'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [19]:
path_to_json = f"C:/Users/{os.environ.get('USERNAME')}/json_api_data.json"
api_key = json.load(open(path_to_json))
api_token = api_key['dev_token']['308']
api_url = api_key['dev_url']
project = redcap.Project(api_url, api_token)

:::

:::

REDCap metadata can be exported via the API. 

The REDCap metadata will contain all the fields in your REDCap project *excluding*:

1) Any survey timestamp or survey identifier fields (automatically generated when a form is enabled as a survey).
2) Any fields that are part of the unique key aside from the `record_id` (`redcap_event_name`, `redcap_repeat_instrument` and `redcap_repeat_instance`).
3) Any `<form_name>_complete` fields.
4) Checkbox fields are exported as one field per checkbox (not in the wide format as they appear in the data export).

In REDCap the `record_id` field can be renamed. The first field in the exported metadata will always be `record_id` or the 'record_id' equivalent field.

::: {.panel-tabset}

#### R

`redcap_metadata_read()`

In [9]:
%%capture
%%R
metadata <- redcap_metadata_read(
    redcap_uri = url, 
    token = token
)$data

::: {.content-hidden when-format="html"}

In [11]:
%%R
metadata_tbl <- gt(head(metadata))
gt::gtsave(metadata_tbl, filename = 'metadata.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="500" src="./files/export_files/metadata.html" title="Quarto Documentation"></iframe></p>

#### Python

`export_metadata()`

In [22]:
project.export_metadata(format_type="df").head(10)

,form_name,section_header,field_type,field_label,select_choices_or_calculations,field_note,text_validation_type_or_show_slider_number,text_validation_min,text_validation_max,identifier,branching_logic,required_field,custom_alignment,question_number,matrix_group_name,matrix_ranking,field_annotation
field_name,,,,,,,,,,,,,,,,,
record_id,demographics,NaN,text,Record ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first_name,demographics,Personal Information,text,First Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last_name,demographics,NaN,text,Last Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
phone_num,demographics,NaN,text,Phone Number,NaN,NaN,phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zip_code,demographics,NaN,text,ZIP Code,NaN,NaN,integer,10001.0,99999,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dob,demographics,NaN,text,Date of birth,NaN,NaN,date_mdy,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,demographics,NaN,calc,Age (years),"rounddown(datediff([dob],'today','y'))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ethnicity,demographics,NaN,radio,Ethnicity,"0, Hispanic or Latino|1, NOT Hispanic or Latin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,LH,NaN,NaN,NaN,NaN
race,demographics,NaN,dropdown,Race,"0, American Indian/Alaska Native|1, Asian|2, N...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


:::